In [2]:
import sys; sys.path.append('..')
import torch
from torchsummary import summary
from src.models.autoencoder import AutoEncoder

DEVICE = 'cuda:1'

ae = AutoEncoder(None).to(DEVICE)
ae.load_state_dict(torch.load('../experiments/lifelong_ae--cifar100--1-00022/checkpoints/model-task-9.pt'))

<All keys matched successfully>

In [3]:
from torchvision.datasets import CIFAR10, CIFAR100
from torchvision.transforms import ToTensor

ds_train = CIFAR100('../data/CIFAR100', train=True, transform=ToTensor())
ds_test = CIFAR100('../data/CIFAR100', train=False, transform=ToTensor())

In [4]:
from torch.utils.data import DataLoader
from torch.utils.data import Subset

# ds_train = Subset(ds_train, range(100))
# ds_test = Subset(ds_test, range(100))

NUM_CLASSES = 100

dl_train = DataLoader(ds_train, batch_size=256, shuffle=True)
dl_test = DataLoader(ds_test, batch_size=256, shuffle=False)

In [5]:
def extract_features(ae, dataloader):
    ae.eval()
    
    y = []
    feats = []
    
    with torch.no_grad():
        for batch in dataloader:
            feats.extend(ae.encoder(batch[0].to(DEVICE)).cpu().flatten(1).tolist())
            y.extend(batch[1])
            
    return feats, y

feats_train, y_train = extract_features(ae, dl_train)
feats_test, y_test = extract_features(ae, dl_test)

In [6]:
feats_dl_train = DataLoader(list(zip(*[feats_train, y_train])), batch_size=256, shuffle=True, collate_fn=lambda b: list(zip(*b)))
feats_dl_test = DataLoader(list(zip(*[feats_test, y_test])), batch_size=256, shuffle=False, collate_fn=lambda b: list(zip(*b)))

In [16]:
len(feats_train)

50000

In [7]:
def validate_clf(clf_model, dataloader):
    losses = []
    accs = []

    with torch.no_grad():
        for x, y in dataloader:
            x = torch.tensor(x).to(DEVICE)
            y = torch.tensor(y).to(DEVICE)

            logits = clf_model(x)
            loss = F.cross_entropy(logits, y, reduction='none').cpu().tolist()
            acc = (logits.argmax(dim=1) == y).float().cpu().tolist()
            
            losses.extend(loss)
            accs.extend(acc)
        
    return np.mean(losses), np.mean(accs)

In [ ]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

classifier = nn.Sequential(
    #nn.Dropout(0.1),
    #nn.Linear(2048, 256),
    #nn.BatchNorm1d(256),
    #nn.ReLU(),
    #nn.Dropout(0.1),
    #nn.Linear(256, 100)
    nn.Linear(512, 512),
    nn.ReLU(),
    #nn.Dropout(0.5),
    nn.BatchNorm1d(512),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 100)
).to(DEVICE)

max_num_epochs = 100
optim = torch.optim.Adam(classifier.parameters(), lr=1e-3)

for epoch in range(max_num_epochs):
    for batch in feats_dl_train:
        x = torch.tensor(batch[0]).to(DEVICE)
        y = torch.tensor(batch[1]).to(DEVICE)

        logits = classifier(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean().detach().cpu()

        optim.zero_grad()
        loss.backward()
        optim.step()

    #print('acc', acc.item())
    if epoch % 5 == 0:
        print('train acc:', validate_clf(classifier, feats_dl_train)[1])
        print('val acc:', validate_clf(classifier, feats_dl_test)[1])

train acc: 0.22108
val acc: 0.203
train acc: 0.34138
val acc: 0.2721
train acc: 0.41652
val acc: 0.2834
train acc: 0.4717
val acc: 0.281
train acc: 0.5171
val acc: 0.2728
train acc: 0.56332
val acc: 0.2679
train acc: 0.60756
val acc: 0.2609
train acc: 0.63146
val acc: 0.258
train acc: 0.66328
val acc: 0.2514
train acc: 0.67984
val acc: 0.2473
train acc: 0.69194
val acc: 0.2471
train acc: 0.71092
val acc: 0.2376
train acc: 0.72816
val acc: 0.2344
train acc: 0.73086
val acc: 0.2335
